In [1]:
# Install Faiss
!pip install faiss-gpu

# Install Pyserini
!pip install pyserini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.9 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:

# Data Prep
!wget https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip -P collections
!unzip collections/nfcorpus.zip -d collections


--2024-02-12 11:03:10--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2448432 (2.3M) [application/zip]
Saving to: ‘collections/nfcorpus.zip’

nfcorpus.zip        100%[===================>]   2.33M  3.01MB/s    in 0.8s    

2024-02-12 11:03:11 (3.01 MB/s) - ‘collections/nfcorpus.zip’ saved [2448432/2448432]

Archive:  collections/nfcorpus.zip
   creating: collections/nfcorpus/
   creating: collections/nfcorpus/qrels/
  inflating: collections/nfcorpus/qrels/train.tsv  
  inflating: collections/nfcorpus/qrels/test.tsv  
  inflating: collections/nfcorpus/qrels/dev.tsv  
  inflating: collections/nfcorpus/corpus.jsonl  
  inflating: collections/nfcorpus/queries.jsonl  


In [3]:

# Query data munging
import json

with open('collections/nfcorpus/queries.tsv', 'w') as out:
    with open('collections/nfcorpus/queries.jsonl', 'r') as f:
        for line in f:
            l = json.loads(line)
            out.write(l['_id'] + '\t' + l['text'] + '\n')

In [4]:



# Relevance judgments (qrels) data munging
!tail -n +2 collections/nfcorpus/qrels/test.tsv | sed 's/\t/\tQ0\t/' > collections/nfcorpus/qrels/test.qrels


In [5]:

# Indexing
!python -m pyserini.encode \
  input   --corpus collections/nfcorpus/corpus.jsonl \
          --fields title text \
  output  --embeddings indexes/faiss.nfcorpus.contriever-msmacro \
          --to-faiss \
  encoder --encoder facebook/contriever-msmarco \
          --device cpu \
          --pooling mean \
          --fields title text \
          --batch 32




config.json: 100% 619/619 [00:00<00:00, 2.61MB/s]
pytorch_model.bin: 100% 438M/438M [00:04<00:00, 105MB/s]
tokenizer_config.json: 100% 321/321 [00:00<00:00, 1.40MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 6.59MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 23.7MB/s]
special_tokens_map.json: 100% 112/112 [00:00<00:00, 463kB/s]
3633it [00:00, 88284.00it/s]
100% 114/114 [48:29<00:00, 25.53s/it]


In [10]:
!apt-get install libomp-dev
!pip install faiss-cpu



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 43.2 MB/s eta 0:00:00


In [11]:
# Retrieval
!python -m pyserini.search.faiss \
  --encoder-class contriever --encoder facebook/contriever-msmarco \
  --index indexes/faiss.nfcorpus.contriever-msmacro \
  --topics collections/nfcorpus/queries.tsv \
  --output runs/run.beir-contriever-msmarco.nfcorpus.txt \
  --batch 128 --threads 16 \
  --hits 1000




Running collections/nfcorpus/queries.tsv topics, saving to runs/run.beir-contriever-msmarco.nfcorpus.txt...
100% 3237/3237 [04:37<00:00, 11.67it/s]


In [12]:
# Evaluation
!python -m pyserini.eval.trec_eval \
  -c -m ndcg_cut.10 collections/nfcorpus/qrels/test.qrels \
  runs/run.beir-contriever-msmarco.nfcorpus.txt

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 5.69MB/s]                
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/nfcorpus/qrels/test.qrels', 'runs/run.beir-contriever-msmarco.nfcorpus.txt']
Results:
ndcg_cut_10           	all	0.3306


In [13]:
from pyserini.search.faiss import FaissSearcher, AutoQueryEncoder

encoder = AutoQueryEncoder('facebook/contriever-msmarco', device='cpu', pooling='mean')
searcher = FaissSearcher('indexes/faiss.nfcorpus.contriever-msmacro', encoder)
hits = searcher.search('How to Help Prevent Abdominal Aortic Aneurysms')

for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.6f}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 1 MED-4555 1.472201
 2 MED-3180 1.125015
 3 MED-1309 1.067153
 4 MED-2224 1.059537
 5 MED-4423 1.038440
 6 MED-4887 1.032622
 7 MED-2530 1.020758
 8 MED-2372 1.016142
 9 MED-1006 1.013599
10 MED-2587 1.010811
